In [1]:
from sqlalchemy import create_engine
import sqlite3
import pandas as pd
from tqdm import tqdm

In [2]:
### Create database connection

engine = create_engine(f'sqlite:///quotes.db', echo=False, connect_args={"timeout": 30})
con = sqlite3.connect('database/quotes.db') # connecting to the database
cursor = con.cursor()

In [3]:
### Get list of tables from database

tables = cursor.execute("SELECT name FROM sqlite_master where type='table';").fetchall() # getting every table name
tables = [table[0] for table in tables]
tables

['Koutetsujou_no_Kabaneri_quotes',
 'Mashle_quotes',
 'Helck_quotes',
 'Dou_Po_Cangqiong_Di_Si_Ji_quotes',
 'Iya_na_Kao_Sarenagara_Opantsu_Misete_Moraitai_quotes',
 'Yubisaki_to_Renren_quotes',
 'Shokugeki_no_Souma_OAD_quotes',
 'Sasaki_to_P-chan_quotes',
 'Kekkon_Yubiwa_Monogatari_quotes',
 'Nozomanu_Fushi_no_Boukensha_quotes',
 'Isekai_Maou_to_Shoukan_Shoujo_no_Dorei_Majutsu_quotes',
 'Isekai_Maou_to_Shoukan_Shoujo_no_Dorei_Majutsu_Omega_quotes',
 'Magical_Hat_quotes',
 'Aho_Girl_quotes',
 'Kaitou_Saint_Tail_quotes',
 'Megami_no_Cafe_Terrace_quotes',
 'World_Trigger_quotes',
 'Kasumin_1st_Series_quotes',
 'Spy_x_Family_(2023)_quotes',
 'Tensei_Shitara_Slime_Datta_Ken_(2021)_quotes',
 'Undead_Unluck_quotes',
 'Pokemon_Concierge_quotes',
 'Yofukashi_no_Uta_quotes',
 'Pon_no_Michi_quotes',
 'Jujutsu_Kaisen_(2023)_quotes',
 'Kage_no_Jitsuryokusha_ni_Naritakute!_2nd_Season_quotes',
 'Shiguang_Dailiren_II_quotes',
 'Pluto_quotes',
 'Watashi_no_Shiawase_na_Kekkon_quotes',
 'Fruits_Basket_1s

In [4]:
### Calculate percentage of unknown character rows

data=[]
for table in tables: # for each table calculate the percentage of unknown character entries
    try:
        unknown_rate=cursor.execute(f"SELECT AVG(is_unknown) \
                                    FROM \
                                        (SELECT Episode, \
                                        (CASE WHEN name IS 'Unknown' THEN 1 ELSE 0 END) AS is_unknown, \
                                        Quote FROM {table}) AS temp").fetchall()
        data.append([table,unknown_rate[0][0]])
    except:
        data.append([table, 'NaN'])

#Write it to the db

df = pd.DataFrame(data=data, columns=['name', 'unknown_rate'])
df.to_sql('tables',con=engine,if_exists='replace', index=False) # load to quotes.db

124

In [5]:
### Clean the database

data=[]
condition = "'%{\pos%'"
for table in tables: # for each table calculate the percentage of unknown character entries
    try:
        brackets_count = cursor.execute(f"SELECT COUNT(*) FROM ( \
                                        SELECT * FROM {table} \
                                        WHERE quote LIKE {condition} \
                                        ) AS temp").fetchall()
        data.append([table,brackets_count[0][0]])
    except:
        data.append([table, 'NaN'])

data

[['Koutetsujou_no_Kabaneri_quotes', 0],
 ['Mashle_quotes', 0],
 ['Helck_quotes', 0],
 ['Dou_Po_Cangqiong_Di_Si_Ji_quotes', 0],
 ['Iya_na_Kao_Sarenagara_Opantsu_Misete_Moraitai_quotes', 0],
 ['Yubisaki_to_Renren_quotes', 0],
 ['Shokugeki_no_Souma_OAD_quotes', 0],
 ['Sasaki_to_P-chan_quotes', 'NaN'],
 ['Kekkon_Yubiwa_Monogatari_quotes', 0],
 ['Nozomanu_Fushi_no_Boukensha_quotes', 0],
 ['Isekai_Maou_to_Shoukan_Shoujo_no_Dorei_Majutsu_quotes', 0],
 ['Isekai_Maou_to_Shoukan_Shoujo_no_Dorei_Majutsu_Omega_quotes', 0],
 ['Magical_Hat_quotes', 0],
 ['Aho_Girl_quotes', 0],
 ['Kaitou_Saint_Tail_quotes', 0],
 ['Megami_no_Cafe_Terrace_quotes', 0],
 ['World_Trigger_quotes', 0],
 ['Kasumin_1st_Series_quotes', 0],
 ['Spy_x_Family_(2023)_quotes', 'NaN'],
 ['Tensei_Shitara_Slime_Datta_Ken_(2021)_quotes', 'NaN'],
 ['Undead_Unluck_quotes', 0],
 ['Pokemon_Concierge_quotes', 0],
 ['Yofukashi_no_Uta_quotes', 0],
 ['Pon_no_Michi_quotes', 0],
 ['Jujutsu_Kaisen_(2023)_quotes', 'NaN'],
 ['Kage_no_Jitsuryokusha_n

In [ ]:
condition = "'%{\pos%'" # condition to delete row
for table in tqdm(tables): # for each table delete quote entries like {\pos} usually they are from opening or name of sub provider
    try:
        result = cursor.execute(f"DELETE FROM {table} WHERE quote LIKE {condition}") # delete quotes where
        con.commit() # commit changes
        print(cursor.rowcount) # get how many rows were affected
    except:
        print('error', table, result)
        pass